<b> Modeling </b> notebook (number 3) for <b>Project 2</b> of General Assembly DSIR-0320 <br>
by Martijn de Vries </br>
martijndevries91@gmail.com

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt

#sklearn imports
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.metrics import mean_squared_error,mean_absolute_error

In [57]:
#load in the feature engineered training data for model 1
df_model= pd.read_csv('../model_inputs/train_engineered_m1.csv')
df_model.head()

,Id,tot_area,gar_adj_area,yr_built,yr_remod,qual,MS Zoning_A (agr),MS Zoning_C (all),MS Zoning_FV,MS Zoning_I (all),MS Zoning_RH,MS Zoning_RM,Func_Maj,Func_Min,Func_Mod,Func_Sev,SalePrice
0,109,2204.0,219.363327,1976,2005,6,0,0,0,0,0,0,0,0,0,0,130500
1,544,3035.0,258.156000,1996,1997,7,0,0,0,0,0,0,0,0,0,0,220000
2,153,2114.0,69.850203,1953,2007,5,0,0,0,0,0,0,0,0,0,0,109000
3,318,1828.0,226.452710,2006,2007,5,0,0,0,0,0,0,0,0,0,0,174000
4,255,2121.0,137.428854,1900,1993,6,0,0,0,0,0,0,0,0,0,0,138500


In [10]:
X = df_model.drop(columns=['Id', 'SalePrice'])
y = df_model['SalePrice']

In [39]:
#I'll use the term 'val' here to not get confused with the actual testing set
X_train, X_val, y_val, y_val= train_test_split(X, y, random_state=42)

In [40]:

lr = LinearRegression()
lr.fit(X_train,y_train)

LinearRegression()

In [41]:
print(lr.score(X_train, y_train))
print(lr.score(X_val, y_val))

0.8274480173385025
0.8577509926637402


That's pretty decent! Clearly the model is not overfit, given the good agreement in scores.

In the feature engineering notebook, we saw that it seems that at least for some predictor variables, the relationship seems more linear to log(SalePrice) rather than SalePrice. What if I fit using that instead?

In [58]:
y_train_log = np.log(y_train)
y_val_log = np.log(y_val)

lr.fit(X_train,y_train_log)

LinearRegression()

In [59]:
print(lr.score(X_train, y_train_log))
print(lr.score(X_val, y_val_log))

0.8785767525740452
0.8524562854447144


Funnily enough, this seems to mainly improve the training score, but the score for the validation set is nearly the same.

In [62]:
#predictions for the log model
preds_val = lr.predict(X_val)
preds_train = lr.predict(X_train)

#RMSE
rmse_val =  mean_squared_error(np.exp(y_val_log), np.exp(preds_val), squared=False)
rmse_train =  mean_squared_error(np.exp(y_train_log), np.exp(preds_train), squared=False)

rmse_train, rmse_val

(27276.273705404798, 24111.320882269487)

In [65]:
def make_kaggle_df(df, r_obj, islog=False):
    """
    With an input dataframe and sklearn regression object
    construct a dataframe for Kaggle submission
    """
    X_test = df.drop(columns=['Id'])
    ids = df['Id']
    
    preds = r_obj.predict(X_test)
    if islog == True:
        preds = np.exp(preds)
    
    d = {'SalePrice': preds}
    kaggle_m1 = pd.DataFrame(data=d, index=ids)
    return kaggle_m1

In [66]:
#Make a first Kaggle submission
df_model_test= pd.read_csv('../model_inputs/test_engineered_m1.csv')

kaggle_m1 = make_kaggle_df(df_model_test, lr, islog=True)

kaggle_m1.to_csv('../datasets/submission_lin_m1.csv')